In [5]:
import numpy as np
import scipy as sp
import pandas as pd

In [6]:
import statsmodels.api as sm

In [7]:
import math
import importlib
moduleName = input()
importlib.import_module(moduleName)

metrics


<module 'metrics' from 'C:\\Users\\jtang\\voyager\\voyager-data-analysis-fa18\\models\\metrics.py'>

In [8]:
moduleName = input()
importlib.import_module(moduleName)

ols


TypeError: '>=' not supported between instances of 'int' and 'NoneType'

In [9]:
# import xlsx data

sales_df = pd.read_excel(r'..\..\data\sales.xlsx')
services_df = pd.read_excel(r'..\..\data\services.xlsx')
survey_df = pd.read_excel(r'..\..\data\survey.xlsx')

In [10]:
# remove any duplicates in the data (does not consider date because some obvious duplicates have slightly different dates)

sales_df = sales_df.drop_duplicates(subset=['customer_id','household_id','model_year', 'model_type', 'msrp', 'trans_type', 'contract_type'])
services_df = services_df.drop_duplicates(subset =['customer_id','household_id', 'datetime'])
survey_df = survey_df.drop_duplicates(subset =['customer_id','household_id', 'datetime'])

In [11]:
sales_df = sales_df.sort_values(by=['customer_id'])
services_df = services_df.sort_values(by=['customer_id'])
survey_df = survey_df.sort_values(by=['customer_id'])

In [12]:
# generate a separate dataframe for each customer

sales_dfs = [x for _, x in sales_df.groupby(sales_df['customer_id'])]
services_dfs = [x for _, x in services_df.groupby(services_df['customer_id'])]
survey_dfs = [x for _, x in survey_df.groupby(services_df['customer_id'])]

In [13]:
sales_dfs[1]

,customer_id,household_id,datetime,model_year,model_class,model_type,msrp,role,trans_type,contract_type
3314,US1000001643054747,5900000000587900,2016-02-05,2015,C,C300W,41175.0,Owner,Pre-owned,Retail
3787,US1000001643054747,5900000000587900,2017-06-24,2016,C,C300W4,51590.0,Owner,Pre-owned,Retail


In [11]:
services_dfs[1]

,customer_id,household_id,datetime,amount_paid,amount_warranty,model_year,model_class,model_type,role
13513,US1000001643054747,5900000000587900,2017-10-23,0.0,0.0,2016,C,C300W4,Owner
13512,US1000001643054747,5900000000587900,2017-08-25,0.0,0.0,2016,C,C300W4,Owner
13511,US1000001643054747,5900000000587900,2018-04-30,0.0,0.0,2016,C,C300W4,Owner
12676,US1000001643054747,5900000000587900,2016-02-23,0.0,0.0,2015,C,C300W,Owner
13509,US1000001643054747,5900000000587900,2017-06-22,0.0,0.0,2016,C,C300W4,Owner
12675,US1000001643054747,5900000000587900,2017-01-20,0.0,0.0,2015,C,C300W,Owner
13506,US1000001643054747,5900000000587900,2017-07-28,0.0,0.0,2016,C,C300W4,Owner
13507,US1000001643054747,5900000000587900,2018-08-28,0.0,0.0,2016,C,C300W4,Owner
13508,US1000001643054747,5900000000587900,2017-06-26,0.0,0.0,2016,C,C300W4,Owner
12677,US1000001643054747,5900000000587900,2016-03-04,0.0,0.0,2015,C,C300W,Owner


In [12]:
sales_dfs[1]

,customer_id,household_id,datetime,model_year,model_class,model_type,msrp,role,trans_type,contract_type
3314,US1000001643054747,5900000000587900,2016-02-05,2015,C,C300W,41175.0,Owner,Pre-owned,Retail
3787,US1000001643054747,5900000000587900,2017-06-24,2016,C,C300W4,51590.0,Owner,Pre-owned,Retail


In [14]:
class Customer:
    classes = []
    release_month = 6 # the month at which next year's model is released (ex: 2018 model releases in June 2017)

    # when the Customer object is instantiated, all its information will be calculated automatically
    def __init__(self, sales_history, service_history, survey_history,
            start_time_ind, end_time_ind, start_time_dep, end_time_dep):
        self.customer_history = sales_history
        # set the time period for which metrics are being calculated, start_time_ind/end_time_ind
        # is the time period for independent variables
        self.start = start_time_ind
        self.end = end_time_ind
        # save the time period for dependent variables
        self.dep_times = (start_time_dep, end_time_dep)
        # calculate total number of transactions a customer has made (helps for calculating other metrics)
        self.total_trans = self.total_transactions()
        # store behavioral metrics in a "summary" list
        self.summary = [
            self.max_purchase(),
            self.min_purchase(),
            self.total_revenue(),
            self.purchase_gap() / self.total_trans,
            self.retail_purchases() / self.total_trans,
        ]
        self.summary.extend(self.purchases_by_class())
        # reset the time period for dependent variables
        self.start, self.end = self.dep_times
        # store dependent metrics in a "response" list
        self.response = [
            self.total_revenue(),
            self.total_transactions() != 0
        ]

    def total_transactions(self):
        total = 0
        for date in self.customer_history['datetime'].values:
            if date >= self.start and date < self.end:
                total += 1
        return total
    def purchase_freq(customer_history, start, end):
        num_trans = 0
        for date in customer_history['DATE'].values:
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                num_trans += 1
            return num_trans
    def max_purchase(customer_history, start, end):
        max_value = 0
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                max_value = max(customer_history['msrp'].values[index], max_value)
        return max_value
    def min_purchase(customer_history, start, end):
        min_value = float('inf')
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                min_value = min(customer_history['msrp'].values[index], min_value)
        return min_value
    def total_revenue(customer_history, start, end):
        total = 0
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                total += customer_history['msrp'].values[index]
        return total

    # Need to confirm this is correct
    def model_purchase_gap(customer_history, start, end):
        release_month = 6
        sum_of_diff = 0
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                purchase_month = encode_month(date)
                model_year = customer_history['model_year'].values[index]
                model_month = (model_year - 1) * 12 + release_month
                sum_of_diff += (purchase_month - model_month)
        return sum_of_diff

    def distinct_classes(customer_history, start, end):
        classes = set()
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                classLetter = customer_history['model_class'].values[index].strip()
                classes.add(classLetter)
        return len(classes)

    def retail_purchases(customer_history, start, end):
        num_retail = 0
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                contract = customer_history['contract_type'].values[index]
                if contract == "Retail": num_retail += 1
        return num_retail

    def purchase_indicator(self):
        return (self.total_trans != 0)


    #compile a dictionary of model to base MSRP data structure to reference
    #(base_msrp, key: mdoel, value: price)

    #Average difference between  base MSRP and transaction MSRP
    def average_value_options(self, start, end):
        #iterate through transaction and access correct base msrp through data structure
        #add total transactions, subtract sum base msrp, divide by #transactions
        return

    #Average amount spent on each servicing transaction
    def spend_per_service(self, start, end):
        service_num = 0
        service_paid = 0
        #for the values in the service history datetime, pick appropriate timeframe
        for index in range(len(self.service_history.values)):
            date = self.service_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                service = self.service_history['AMT_DMS_CP_STOT'].values[index]
                warranty = self.service_history['AMT_DMS_WARR_STOT'].values[index]
                service_paid = service_num + service + warranty
                service_num += 1
        return service_paid/service_num

    #Average difference between successive vehicle purchase MSRPs
    def change_vehicle_spend(self, start, end):
        differences = []
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                if customer_history['contract_type'].values[index] == "Retail" and index < len(customer_history) - 1:
                    prev = customer_history['AMT_TOT_MSRP'].values[index]
                    cur = customer_history['AMT_TOT_MSRP'].values[index + 1]
                    differences.append((prev-cur))
        return sum(differences)/self.total_trans

    #Total number of X-class vehicles purchased
    #dictionary for each X-class. kinda clunky but should work
    def total_class_purchase(self, start, end):
        class_totals = dictionary()
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                if  customer_history['contract_type'].values[index] == "Retail":
                    model = customer_history['MODEL_CLASS'].values[index]
                    if model not in class_totals:
                        class_totals.update(model, 1)
                    else:
                        class_totals.update(model, class_totals.get(model) + 1)
        return class_totals

    #Number of household vehicles serviced within the last year
    #maybe wrong, this is total services used within a time interval
    def active_household_inventory(self, start, end):
        service_total = 0
        for date in self.service_history['datetime'].values:
            if date >= start and date < end:
                service_total += 1
        return total

    #Average amount of time between individual vehicle purchases
    #How did we say we were doing datetime?
    def average_vehicle_interval(self, start, end):
        count = 0
        holder = list()
        differences = list()
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                if customer_history['contract_type'].values[index] == "Retail":
                    holder.append(encoded)
                    count +=1
        for i in len(holder)-1:
            differences.append(holder[i+1] - holder[i])
        return sum(differences)/count

    #Average number of transactions per year
    #need to re-write to go by year
    def annual_service_freq(self, start, end):
        service_total = 0
        for date in self.service_history['datetime'].values:
            if date >= start and date < end:
                service_total += 1
        return service_total

    #Ratio of leasing transactions to total transactions
    def lease_rate(self, start, end):
        lease_count = 0
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            if date >= start and date < end:
                if customer_history['contract_type'].values[index] == "lease":
                    lease_count += 1
        return lease_count/self.total_trans

    #Ratio of time since last purchase versus average purchase interval
    def recency_score(self, today_date):
        last_purchase_date = 0
        today = encode_date(today_date)
        avgerage_interval = average_vehicle_interval(start, end)
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            encoded = encode_date(date)
            last_purchase_date = max(last_purchase_date, encoded)
        return (today - last_purchase_date)/average_interval

    #Number of intervals between servicing transactions that exceed X years
    def aggregate_service_inactivity(self, start, end, x_years):
        holder = list()
        differences = list()
        for index in range(len(self.service_history.values)):
            date = self.service_history['datetime'].values[index]
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                holder.append(encoded)
        for i in len(holder)-1:
            differences.append(holder[i+1] - holder[i])
        return len([y for y in differences if y > x_years])

    #Average difference between year of purchase and model year of vehicle
    def year_disparity(self):
        differences = list()
        for index in range(len(customer_history.values)):
            date = customer_history['datetime'].values[index]
            purchase_year = encode_year(date)
            model_year = customer_history['MODEL_YEAR'].values[index]
            differences.append(purchase_year - model_year)
        return sum(differences)/self.total_trans

    #Index reflecting consumer sentiment and buying intentions
    def cci(self):
        return

    #Average annual disposable income
    def disposable_income(self):
        return


In [16]:
c1 = Customer(sales_df[1], service_dfs[1], survey_dfs[1], None,None,None,None)

KeyError: 1

In [18]:
# Calculate how many customers have purchased x cars from the start to end dates of data,
# where x is the index of the list "count"

count = [0]*100
for customer in sales_dfs:
    num_trans = len(customer)
    count[num_trans] += 1
for index in reversed(range(0,len(count)-1)):
    count[index] += count[index+1]
count[0] = "NA"
print(count)

['NA', 5000, 1814, 861, 466, 250, 142, 81, 59, 40, 30, 17, 13, 12, 9, 6, 5, 5, 5, 5, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
def encode_year(date_str):
    date = date_str.split('/')
    return int(date[2])
def encode_month(date_str):
    date = date_str.split('/')
    return int(date[0]) + int(date[2])*12
def encode_date(date_str):
    date = date_str.split('/')
    return (int(date[0]) - 1)*31 + (int(date[1]) - 1) + int(date[2])*372

In [22]:
def purchase_freq(customer_history, start, end):
        num_trans = 0
        for date in customer_history['DATE'].values:
            encoded = encode_date(date)
            if encoded >= start and encoded < end:
                num_trans += 1
            return num_trans

In [23]:
# testing
sales_dfs[4]

,customer_id,household_id,datetime,model_year,model_class,model_type,msrp,role,trans_type,contract_type
2420,US1000001643104169,5900000004212930,2017-06-17,2015,GL,GL350BTC,78875.0,Owner,Pre-owned,Retail


In [24]:
services_dfs[4]

,customer_id,household_id,datetime,amount_paid,amount_warranty,model_year,model_class,model_type,role
2049,US1000001643122083,5900000000688030,2010-08-30,0.00,0.00,2008,M,ML350,Owner
2033,US1000001643122083,5900000000688030,2010-06-12,0.00,0.00,2008,M,ML350,Owner
2055,US1000001643122083,5900000000688030,2013-01-16,141.21,0.00,2008,M,ML350,Owner
2056,US1000001643122083,5900000000688030,2008-11-11,0.00,172.50,2008,M,ML350,Owner
2057,US1000001643122083,5900000000688030,2015-07-18,0.00,0.00,2008,M,ML350,Owner
2058,US1000001643122083,5900000000688030,2015-08-28,0.00,0.00,2008,M,ML350,Owner
2065,US1000001643122083,5900000000688030,2008-11-10,0.00,0.00,2008,M,ML350,Owner
2064,US1000001643122083,5900000000688030,2008-11-24,0.00,0.00,2008,M,ML350,Owner
2063,US1000001643122083,5900000000688030,2015-10-27,0.00,0.00,2008,M,ML350,Owner
2062,US1000001643122083,5900000000688030,2016-01-27,0.00,0.00,2008,M,ML350,Owner


In [25]:
survey_dfs[4]

,customer_id,household_id,datetime,disregard,ltr,survey_type,model_year,model_class,model_type,role
1255,US1000001712724521,5900000004107670,2017-10-28 23:04:27,0,1.0,SV,2017,GLA,GLA250W4,Owner


In [26]:
# same counter as before, except smaller date range to account for lack of 2003, 2004 data

x_1 = encode_date("10/23/2005")
x_2 = encode_date("10/23/2018")
count_x = [0]*39

for customer in sales_dfs:
    freq = purchase_freq(customer, x_1, x_2)
    count_x[freq] += 1
for index in reversed(range(0,len(count)-1)):
    count_x[index] += count_x[index+1]
count_x[0] = "NA"
print(count_x)


KeyError: 'DATE'

In [1]:
#Generates list of customers that purchased at least one car in 2 disjoint (or overlapping) date ranges

x_1 = metrics.encode_date("10/23/2005")
x_2 = metrics.encode_date("10/23/2011")
count_x = [0]*39
subset = []
for customer in sales_dfs:
    freq = metrics.purchase_freq(customer, x_1, x_2)
    if freq != 0:
        subset.append(customer['CUST_ID'].values[0])
    count_x[freq] += 1
for index in reversed(range(0,len(count)-1)):
    count_x[index] += count_x[index+1]
count_x[0] = "NA"
print(count_x)
print(len(subset))

x_1 = metrics.encode_date("10/23/2005")
x_2 = metrics.encode_date("10/23/2011")
count_x = [0]*39
subset2 = []
for customer in sales_dfs:
    freq = metrics.purchase_freq(customer, x_1, x_2)
    if freq != 0:
        subset2.append(customer['CUST_ID'].values[0])
    count_x[freq] += 1
for index in reversed(range(0,len(count)-1)):
    count_x[index] += count_x[index+1]
count_x[0] = "NA"
print(count_x)
print(len(subset2))
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3
subset = intersection(subset,subset2)


NameError: name 'metrics' is not defined

In [34]:
# Collecting customer metrics from start to end date
# Calculating revenue from rev_start to rev_end date

customer_summary = dict()
start    = encode_date("10/23/2008")
end      = encode_date("10/23/2013")
resp_start    = ncode_date("10/23/2013")
resp_end      = encode_date("10/23/2018")
interval = 9

customer = sales_dfs[0]
customer = Customer(customer, "NOT IMPLEMENTED", "NOT IMPLEMENTED", 
                   start, end, resp_start, resp_end)
print(customer.summary)

for customer in sales_dfs:
    customer_ID = customer['customer_id'].values[0]
    if customer_ID in subset:
        summary = []
        total_purchases = purchase_freq(customer, start, end)
        summary.append(
            total_purchases / interval
        )
        if purchase_freq(customer, start, end) == 0: 
            customer_summary[customer_ID] = 0
            continue
        summary.append(
            max_purchase(customer, start, end)
        )
        summary.append(
            min_purchase(customer, start, end)
        )
        summary.append(
            model_purchase_gap(customer, start, end) / total_purchases
        )
        summary.append(
            distinct_classes(customer, start, end) / total_purchases
        )
        summary.append(
            retail_purchases(customer, start, end) / total_purchases
        )
        revenue = total_revenue(customer, resp_start, resp_end)
        purchase = purchase_indicator(customer, resp_start, resp_end)
        customer_summary[customer_ID] = (summary, purchase)
    

NameError: name 'encode_date' is not defined

In [26]:
# Generates a data frame consisting of all parameters (independent and dependent)

customer_summary_df = []
for summary in customer_summary.values():
    if summary != 0 and  not math.isnan(summary[1]):
        customer_summary_df.append(summary[0]+[summary[1]])

X = pd.DataFrame(np.array(customer_summary_df), columns = ['freq','max','min','enthu', 'classes','retail',  'purchase'])
X

,freq,max,min,enthu,classes,retail,purchase
0,0.111111,34855.0,34855.0,55.000000,1.000000,1.000000,0.0
1,0.111111,51440.0,51440.0,39.000000,1.000000,1.000000,0.0
2,0.111111,93270.0,93270.0,16.000000,1.000000,0.000000,0.0
3,0.111111,57675.0,57675.0,12.000000,1.000000,1.000000,0.0
4,0.222222,105485.0,71295.0,0.000000,1.000000,0.000000,1.0
5,0.222222,64100.0,38010.0,2.000000,1.000000,0.000000,1.0
6,0.111111,56525.0,56525.0,4.000000,1.000000,0.000000,0.0
7,0.111111,56040.0,56040.0,5.000000,1.000000,1.000000,1.0
8,0.111111,65860.0,65860.0,21.000000,1.000000,1.000000,0.0
9,0.111111,53415.0,53415.0,0.000000,1.000000,1.000000,1.0


In [27]:
# runs regression with specified dependent variables

est = sm.OLS(X['purchase'], X[['freq','enthu', 'classes']]).fit()

est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               purchase   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.545
Method:                 Least Squares   F-statistic:                     513.5
Date:                Wed, 31 Oct 2018   Prob (F-statistic):          4.71e-219
Time:                        18:53:52   Log-Likelihood:                -806.97
No. Observations:                1284   AIC:                             1620.
Df Residuals:                    1281   BIC:                             1635.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
freq           1.3656      0.098     13.906      0.000       1.173       1.558
enthu         -0.0083      0.001    -13.810      0.000      -0.010      -0.007
classes        0.3514      0.024     14.925      0.000       0.305       0.398
==============================================================================
Omnibus:                      532.935   Durbin-Watson:                   1.870
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               66.858
Skew:                           0.045   Prob(JB):                     3.03e-15
Kurtosis:                       1.886   Cond. No.                         198.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""